### Задание 1

In [8]:
import pandas as pd
import zipfile
import json
import sqlite3
from pathlib import Path

In [2]:
# Читаем файл json и сохраняем в DataFrame
okved_df = pd.read_json('okved_2.json.zip')

In [3]:
okved_df

,code,parent_code,section,name,comment
0,01,A,A,"Растениеводство и животноводство, охота и пред...",Эта группировка включает:\n- два основных вида...
1,01.1,01,A,Выращивание однолетних культур,Эта группировка включает:\n- выращивание однол...
2,01.11,01.1,A,"Выращивание зерновых (кроме риса), зернобобовы...",Эта группировка включает:\n- все формы выращив...
3,01.11.1,01.11,A,Выращивание зерновых культур,
4,01.11.11,01.11.1,A,Выращивание пшеницы,
...,...,...,...,...,...
2813,Q,,Q,ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ ЗДРАВООХРАНЕНИЯ И СОЦИА...,Этот раздел включает:\n- предоставление деятел...
2814,R,,R,"ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ КУЛЬТУРЫ, СПОРТА, ОРГАН...",Данный раздел включает:\n- широкий спектр видо...
2815,S,,S,ПРЕДОСТАВЛЕНИЕ ПРОЧИХ ВИДОВ УСЛУГ,Этот раздел включает:\n- деятельность обществе...
2816,T,,T,ДЕЯТЕЛЬНОСТЬ ДОМАШНИХ ХОЗЯЙСТВ КАК РАБОТОДАТЕЛ...,


In [4]:
okved_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2818 entries, 0 to 2817
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         2818 non-null   object
 1   parent_code  2818 non-null   object
 2   section      2818 non-null   object
 3   name         2818 non-null   object
 4   comment      2818 non-null   object
dtypes: object(5)
memory usage: 110.2+ KB


In [5]:
# Создаём БД hw1.db и таблицу okved в ней
create_stmt = \
"""
create table if not exists okved(
    code text,
    parent_code text,
    section text,
    name text,
    comment text
)
"""
with sqlite3.connect('hw1.db') as connection:
    cursor = connection.cursor()
    try:
        cursor.execute(create_stmt)
    except(Exception) as error:
        print(error)
        connection.rollback()
    else:
        connection.commit()
    finally:
        cursor.close()

In [6]:
# Записываем данные из DataFrame в БД
with sqlite3.connect('hw1.db') as connection:
    okved_df.to_sql('okved', connection, if_exists='append', index=False, method='multi')

In [7]:
# Проверяем получившийся результат
with sqlite3.connect('hw1.db') as connection:
    df = pd.read_sql('select * from okved', connection)

display(df)

,code,parent_code,section,name,comment
0,01,A,A,"Растениеводство и животноводство, охота и пред...",Эта группировка включает:\n- два основных вида...
1,01.1,01,A,Выращивание однолетних культур,Эта группировка включает:\n- выращивание однол...
2,01.11,01.1,A,"Выращивание зерновых (кроме риса), зернобобовы...",Эта группировка включает:\n- все формы выращив...
3,01.11.1,01.11,A,Выращивание зерновых культур,
4,01.11.11,01.11.1,A,Выращивание пшеницы,
...,...,...,...,...,...
2813,Q,,Q,ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ ЗДРАВООХРАНЕНИЯ И СОЦИА...,Этот раздел включает:\n- предоставление деятел...
2814,R,,R,"ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ КУЛЬТУРЫ, СПОРТА, ОРГАН...",Данный раздел включает:\n- широкий спектр видо...
2815,S,,S,ПРЕДОСТАВЛЕНИЕ ПРОЧИХ ВИДОВ УСЛУГ,Этот раздел включает:\n- деятельность обществе...
2816,T,,T,ДЕЯТЕЛЬНОСТЬ ДОМАШНИХ ХОЗЯЙСТВ КАК РАБОТОДАТЕЛ...,


In [10]:
df[df.name.str.lower() == 'деятельность в сфере телекоммуникаций']

,code,parent_code,section,name,comment
2193,61,J,J,Деятельность в сфере телекоммуникаций,Эта группировка включает:\n- деятельность по п...


### Задание 2

In [1]:
import zipfile
import sqlite3
import pandas as pd
from pathlib import Path
# from tqdm import tqdm
# from tqdm.notebook import tqdm_notebook
from multiprocessing.pool import ThreadPool as Pool
# from concurrent.futures import ThreadPoolExecutor, as_completed
import os
# from datetime import datetime
from typing import List
import multiprocutils as mpu

In [11]:
file_path = Path('C:', 'Users', 'greym', 'Downloads', 'hw1', 'egrul.json.zip')


def splitter(iterable: 'iterable', batchsize: int = 8) -> list:
    """Разбивает список на фрагменты и создаёт список списков
       с заданным в batcksize размером фрагмента"""
    import numpy as np
    d, m = divmod(len(iterable), batchsize)
    if not d:
        return [iterable]
    chunked_lst = np.array(iterable[:-m]).reshape(d, batchsize).tolist()
    chunked_lst.append(iterable[-m:])
    return chunked_lst


def parser_json(json_file: str) -> pd.DataFrame:
    """Парсит один файл JSON"""
    with zipobj.open(json_file) as file:
        return pd.read_json(file)


with zipfile.ZipFile(file_path, 'r') as zipobj:
    files = zipobj.namelist()
    df = parser_json(files[0])

In [3]:
executor = ThreadPoolExecutor(max_workers=4)
executor

In [ ]:
executor.submit()

In [13]:
processor_df(df)

,ogrn,inn,kpp,name,full_name,code_okved,name_okved,type_okved
169,1114910000480,4909109443,490901001,"ООО ""ИТЛАЙН""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИТЛАЙН""",61.10,Деятельность в области связи на базе проводных...,Осн


In [11]:
chunked_files[0]

['06439.json',
 '04669.json',
 '03534.json',
 '00393.json',
 '07388.json',
 '10339.json',
 '09162.json',
 '05575.json',
 '06849.json',
 '08312.json',
 '05296.json',
 '08667.json',
 '10549.json',
 '10851.json',
 '03216.json',
 '07553.json',
 '10245.json',
 '08935.json',
 '00981.json',
 '07602.json',
 '07293.json',
 '08377.json',
 '10193.json',
 '08450.json',
 '06832.json',
 '03937.json',
 '05046.json',
 '10364.json',
 '02313.json',
 '06268.json',
 '04600.json',
 '03075.json',
 '05820.json',
 '06266.json',
 '04874.json',
 '11118.json',
 '10194.json',
 '08771.json',
 '00407.json',
 '03811.json',
 '08657.json',
 '10723.json',
 '03702.json',
 '06595.json',
 '05163.json',
 '09753.json',
 '00925.json',
 '11196.json',
 '02636.json',
 '07499.json',
 '00204.json',
 '09795.json',
 '09065.json',
 '05850.json',
 '10004.json',
 '06116.json',
 '10409.json',
 '08251.json',
 '11441.json',
 '03714.json',
 '02053.json',
 '05884.json',
 '05410.json',
 '03382.json',
 '09521.json',
 '02268.json',
 '06109.js

In [14]:
import os
from pathlib import Path
next(Path('C:').glob('**/Users/greym/Downloads/egrul.json.zip'))

KeyboardInterrupt: 

In [15]:
from glob import glob

In [29]:
glob('C:Users/greym/Downloads/hw1/egrul.json.zip', recursive=True)

['C:Users/greym/Downloads/hw1/egrul.json.zip']

In [21]:
os.getcwd()

'D:\\Курсы\\Advanced Python (MIPT)\\hw1'

In [24]:
Path.home()

WindowsPath('C:/Users/greym')

In [33]:
Path('C:/', 'Users', 'greym', 'Downloads', 'hw1', 'egrul.json.zip')

WindowsPath('C:/Users/greym/Downloads/hw1/egrul.json.zip')

In [34]:
for i in range(1):
    print(i)

0


In [6]:
file_path = Path('C:', 'Users', 'greym', 'Downloads', 'hw1', 'egrul.json.zip')

def extract(path: 'str | Path') -> List[str]:
    with zipfile.ZipFile(path, 'r') as zipobj:
        return zipobj.namelist()


# Извлекаем список файлов из архива .zip
files = extract(file_path)
len(files) / 3

3819.0

In [ ]:
NPROC = os.cpu_count() - 1  # количество процессоров
MAX_WORKERS = min(32, os.cpu_count() + 4)  # количество потоков

file_path = Path('C:', 'Users', 'greym', 'Downloads', 'hw1', 'egrul.json.zip')

# def splitter(iterable: 'iterable', batchsize: int = 8) -> list:
#     """Разбивает список на фрагменты и создаёт список списков
#        с заданным в batcksize размером фрагмента"""
#     import numpy as np
#     d, m = divmod(len(iterable), batchsize)
#     if not d:
#         return [iterable]
#     chunked_lst = np.array(iterable[:-m]).reshape(d, batchsize).tolist()
#     chunked_lst.append(iterable[-m:])
#     return chunked_lst


def extract(path: 'str | Path') -> List[str]:
    with zipfile.ZipFile(path, 'r') as zipobj:
        return zipobj.namelist()


# Извлекаем список файлов из архива .zip
files = extract(file_path)
# рассчитываем chunksize
chunksize, extra = divmod(len(files), NPROC * 4)
if extra:
    chunksize += 1

# Создаём генератор-парсер данных из .zip архива в формате JSON
gen_dfs = mpu.unpacker(file_path, files, 3)

# Парсим данные из JSON файлов в DF в параллельном режиме
# with Pool(processes=NPROC) as process_pool:
#     dfs = process_pool.map(mpu.processor_df, gen_dfs, 3819)

dfs = mpu.mulitproc(gen_dfs, NPROC, 3819)

  2%|█▏                                                                          | 177/11457 [01:15<1:26:33,  2.17it/s]

In [ ]:
# Объединяем все данные по всем юр. лицам в один DF
egrul_df = pd.concat(dfs, ignore_index=True)

17it [22:47, 81.24s/it]

In [34]:
# Удаляем лишний столбец data
egrul_df.drop(columns='data', inplace=True)

In [35]:
egrul_df

,ogrn,inn,kpp,name,full_name,code_okved,name_okved,type_okved
0,1114910000480,4909109443,490901001,"ООО ""ИТЛАЙН""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИТЛАЙН""",61.10,Деятельность в области связи на базе проводных...,Осн
1,1127847488053,7811531478,781101001,"ООО ""СМО СИСТЕМА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СМО ...",61.30,Деятельность в области спутниковой связи,Осн
2,1197536005787,7536178239,753601001,"ООО ""ИНТЕР КОД""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНТЕ...",61.90,Деятельность в области телекоммуникаций прочая,Осн
3,1197536005919,7513007283,751301001,"ООО ""ВОНГОЛА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОНГ...",61.10,Деятельность в области связи на базе проводных...,Осн
4,1197627000691,7604352760,760401001,"ООО ""РЭС""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РИНГ...",61.10,Деятельность в области связи на базе проводных...,Осн
...,...,...,...,...,...,...,...,...
20912,1111902001694,1902024322,190201001,"ООО ""САЯНСВЯЗЬ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САЯН...",61.10.1,Деятельность по предоставлению услуг телефонно...,Осн
20913,1112031001422,2016001613,201601001,"ООО ""АВТО-НАВИГАТОР""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АВТО...",61.1,Деятельность в области связи на базе проводных...,Осн
20914,1112031002038,2014005080,201401001,"ООО ""ГАРАНТТОРГ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГАРА...",61.10,Деятельность в области связи на базе проводных...,Осн
20915,1112031003831,2014005877,201401001,"ООО ""РЕАС""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕАС""",61.10,Деятельность в области связи на базе проводных...,Осн


In [36]:
# Создаём таблицу telecom_companies БД hw1.db
create_stmt = \
"""
create table if not exists telecom_companiesokved(
    inn integer,
    ogrn integer,
    kpp integer,
    name text,
    full_name text,
    code_okved text,
    name_okved text,
    type_okved text
)
"""
with sqlite3.connect('hw1.db') as connection:
    cursor = connection.cursor()
    try:
        cursor.execute(create_stmt)
    except(Exception) as error:
        print(error)
        connection.rollback()
    else:
        connection.commit()
    finally:
        cursor.close()

In [37]:
# Записываем данные из DataFrame в БД
with sqlite3.connect('hw1.db') as connection:
    egrul_df.to_sql('telecom_companiesokved', connection,
                    if_exists='append',
                    index=False,
                    method='multi')

In [9]:
# Проверяем получившийся результат
with sqlite3.connect('hw1.db') as connection:
    df = pd.read_sql('select * from telecom_companiesokved', connection)

display(df)

,inn,ogrn,kpp,name,full_name,code_okved,name_okved,type_okved
0,7838003370,1037861007799,470501001,"ООО ""РЭЙС СЕВЕРО-ЗАПАД""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЭЙС...",61.10.1,Деятельность по предоставлению услуг телефонно...,Осн
1,4909109443,1114910000480,490901001,"ООО ""ИТЛАЙН""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИТЛАЙН""",61.10,Деятельность в области связи на базе проводных...,Осн
2,2707006265,1022700926896,272101001,"КГУП ""ХАБАРОВСКВОДТРАНС""",КРАЕВОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ...,61.20,Деятельность внутреннего водного транспорта,Осн
3,2721099060,1022700931912,272501001,"ООО ""ЭЛКОМ ДВ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛКО...",61.10,Деятельность в области связи на базе проводных...,Осн
4,6686060442,1156686002263,667001001,"ООО ""ЕК-ПРЕСТИЖ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЕК-П...",61.10,Деятельность в области связи на базе проводных...,Осн
...,...,...,...,...,...,...,...,...
16584,6317147081,1206300031541,631701001,"ООО ""АГЕНТСТВО БЕЗОПАСНОСТИ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АГЕН...",61.10.9,Деятельность в области связи на базе проводных...,Осн
16585,6317147476,1206300034930,631701001,"ООО ""ООО СВЯЗНОЙ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ООО ...",61.10,Деятельность в области связи на базе проводных...,Осн
16586,6317147525,1206300035281,631701001,"ООО ""ТЕЛЕКОММУНИКАЦИИ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТЕЛЕ...",61.10,Деятельность в области связи на базе проводных...,Осн
16587,2315144585,1082315004353,231501001,"ООО ""ПАЛЛАДА ШИППИНГ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПАЛЛ...",61.10,Деятельность морского транспорта,Осн
